# Service Industry

A small spreadsheet puclished by the UK Office for National Statistcs, makes heavy use of whitespace and inconsistent and fairly irregular spatial relationships between concepts. There's also some fairly semantically lackluster things scattered throughout, i.e `<date> ->` (see cell A21).

So while neither particuarly big nor structually complex there are a few processing steps needed to make this source legible as tidy data.

_Note - you could go into quite some detail capturing nuances of footnotes against values if needed, for our purposes here I'm keeping in the bounds of extracting the basic data table_.

## Tutorial Structure

With these example tutorials I'm going to comment heavily and cover nuances in a follow up section (with liberal targetted previews as needed) as it's the easiest way to grapple with new ideas. It may also be worth opening up these notebooks yourself (they're in `./jupyterbook` in the [tidychef](https://github.com/mikeAdamss/tidychef) github repo) so you can run, alter and generally have a play about with this yourself as part of your learning.

We'll cover:

- source data
- requirements, what we're aiming to do here
- show the full script (all logic commented)
- output the selection preview
- nuances (where applicable)
- view the output

This sequencing is necessary as the output for some of the example is **really** long so that necessitates it coming last. If you're viewing this via a jupyter book (i.e on the site) you can navigate between the above sections via your right hand menu.

_Note - these tutorial scripts might seem verbose due all the comments but that's ok (this is a tutorial after all). If you take them out you end up with a fairly succinct and human readable encapsulation of what would otherwise (with existing tools) be a rather convoluted and fragile set of instructions to express._

In virtually all cases I'll make heavy use of `preview` and `bounded` to only look at relevant parts of what can be quite large datasets. Downloads links are provided for the source data.

## Source

For this example we're extracting the table "TOPS19" as shown below (note - preview cropped for reasons of practicality):

In [ ]:
from tidychef import acquire, preview
from tidychef.selection import XlsSelectable

table: XlsSelectable = acquire.xls.http("https://raw.githubusercontent.com/mikeAdamss/tidychef/main/tests/fixtures/xls/service-industry.xls", tables="TOPSI9")
preview(table, bounded="A1:Q22")


From an xlsx source which can be [downloaded here](https://raw.githubusercontent.com/mikeAdamss/tidychef/main/tests/fixtures/xls/service-industry.xls).

# Requirements

- We'll take the line 4 and 5 headers as "Production".
- Time is an add amalgamation of columns A+B so we'll tidy up and concatentate them.
- We'll take row 9 as "CDID" (as I happen to know that's the name of this particular type of identifier).
- We'll take the main observations as "Observations"
- We'll clean off the sub table at the bottom as unneccesary for our purpose here.

In [ ]:
from typing import List
from tidychef import acquire, against, preview, filters
from tidychef.direction import up, down, left, right
from tidychef.output import Column, TidyData
from tidychef.selection import XlsSelectable

table: XlsSelectable = acquire.xls.http("https://raw.githubusercontent.com/mikeAdamss/tidychef/main/tests/fixtures/xls/service-industry.xls", tables="TOPSI9")

# We have an extended footer section (almost a separate table) under the main table, we're gonna select it here
# just so we can remove it from the other selections
unwanted = table.excel_ref('A').filter(filters.contains_string("Average")).expand(right).expand(down)

# - (a) Find the row with a cell containing the string "ships"
# - (b) Extrude (expand but only one row) the selection down
# - (c) Extrude up we now have the ships row along with the row above and below it
# - (d) remove any blank cells from the selection
product = table.cell_containing_string("ships",strict=False).extrude(down).extrude(up).expand(right).is_not_blank().label_as("Producton")

# -----------------
# Approach to time:
# We're gonna take time as "year" and "time_qualifier" (quarter, month or blank) and concatentate them
# in the TidyData class, to give us a "Time" column of time_qualifer+year e.g '2019', 'Q1 2019', 'Jul 2019'.
# -----------------

# Get numeric values from column A.Expand down from them (to sweep up the odd -> annotations) and remove
# anything we might have swept up from the footer section
year = table.column('A').is_numeric().expand(down).is_not_blank().label_as("Year") - unwanted

# We'll cover this in nuances
time_qualifier = year.shift(right).expand(down).is_not_blank().label_as("Time Qualifier") - unwanted | year.shift(right)

# Get rows 8-10 then use a regex to find the CDID's (cells with four upper case characters)
# We're taking the rows 8,10 (currently wither side of the row we want) as a bit of future proofing
# against layout changes.
cdid = table.excel_ref('8:10').re(r"^[A-Z]{3}\d$").assert_single_row().label_as("CDID")

# Use a waffle to the observations.
observations = cdid.waffle(down, time_qualifier).is_not_blank().label_as("Observations")

# Create a bounded preview inline but also write the full preview to path
preview(product, year, time_qualifier, cdid, observations)

tidy_data = TidyData(
    observations,
    Column(product.attach_directly(down)),
    Column(year.attach_closest(down), apply=lambda x: x[:4], validate=against.is_numeric), # lets both strip and make sure we've stripped the '->'
    Column(time_qualifier.attach_directly(right)),
    Column(cdid.attach_directly(down)),
    Column.horizontal_condition("Time", lambda x: (x["Time Qualifier"] + " " + x["Year"]).strip()),
    drop=["Year", "Time Qualifier"]
)

tidy_data.to_csv("service-industry.csv")

## Nuances

### Extracting the time qualifer

I mainly wanted to detail this as it's a good example of using union during an extraction to handle an awkward scenario. Here's the line in question:

```python
time_qualifier = year.shift(right).expand(down).is_not_blank().label_as("time2") - unwanted | year.shift(right)
```

Now we'll step through what's actually happening here.

In [ ]:
# Lets get the left side of the selection
examp1e1 = year.shift(right).expand(down).is_not_blank().label_as("Time Qualifer - left side of union") - unwanted
preview(examp1e1, bounded="A5:H22")

So this gives us **some but not all** of the selections we need.

Notice we're missing a selection for B11:B15. All columns **must** have a cell to lookup, even if that cell is blank (blank is fine `"" + year` just ends up with year, which is the right result in this scenario). So we need to find a way to add those missing four blank cells to the selection.

Which brings us the right side of the union statement - lets see what it resolves to.

In [ ]:
right_side_of_union = year.shift(right).label_as("Time Qualifer - right side of union") 
preview(right_side_of_union, bounded="A5:H22")

So this _also_ gives us **some but not all**  of what we need.

So we need _both_, so (to bring us full circle) we need a `union` of the cells in these selctions, hence:

In [ ]:
time_qualifier = year.shift(right).expand(down).is_not_blank().label_as("Time Qualifier") - unwanted | year.shift(right)
preview(time_qualifier, bounded="A5:H22")

# Outputs

The tidy data can be [view here](./service-industry.csv) and a full inline preview of the tidydata generated is shown below for those people who'd prefer to scroll.

In [ ]:
print(tidy_data)